In [1]:
import pandapower as pp
import pandapower.networks as nw
import pandas as pd

# Step 1: Define the power system
net = nw.example_simple()  # This would be replaced with a real network definition

# Step 2: Run initial load flow
initial_res = pp.runpp(net)

# Step 3: Define objective function - Minimize losses
def objective_function(net):
    pp.runpp(net)
    losses = sum(net.res_line.p_mw)  # Total real power loss in the lines
    return losses

# Step 4: Define constraints - Voltage limits, line thermal limits, etc.
def constraints(net):
    voltages_ok = all(net.res_bus.vm_pu.between(0.95, 1.05))
    lines_ok = all(net.res_line.loading_percent <= 100)
    return voltages_ok and lines_ok

# Step 5: Use an optimization algorithm
def optimize_topology(net):
    # For the example, let's consider switching operations
    for switch in net.switch.index:
        if constraints(net):
            net.switch.at[switch, 'closed'] = not net.switch.at[switch, 'closed']
            if not constraints(net):  # If the change is not feasible, revert it
                net.switch.at[switch, 'closed'] = not net.switch.at[switch, 'closed']
            else:
                # Calculate the objective function with the new state
                new_losses = objective_function(net)
                # If the new state is better, keep it, otherwise revert it
                # This decision logic would be more complex in a real case
    return net

# Optimize the network
optimized_net = optimize_topology(net)


AttributeError: 'DataFrame' object has no attribute 'p_mw'